In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
if '__file__' not in globals(): __file__, __name__ = globals()['__vsc_ipynb_file__'], '__ipynb__'

import sys, os
if os.path.abspath('.') not in sys.path: sys.path.append(os.path.abspath('.'))


### with sys.profile

In [ ]:
import sys
import threading
import time

def _getfi(frame):
    if frame is None: return ('?', '?', -1)

    func_name = frame.f_code.co_name
    if 'self' in frame.f_locals:
        class_name = frame.f_locals['self'].__class__.__name__
        func_name = class_name + '.' + func_name

    src_path = frame.f_code.co_filename
    # src_path = os.path.basename(frame.f_code.co_filename)
    return (func_name, src_path, frame.f_lineno)


indent = ''
def cb_trace(frame, event, arg):
    global indent
    
    if "/tmp/ipykernel" not in frame.f_code.co_filename: 
        return # return if not notebook src.
    
    func, srcpath, line = _getfi(frame)
    if event == 'call': 
        print(indent, '>', func, srcpath, line, threading.current_thread().name)
        indent += '\t'
    
    elif event == 'return': 
        indent = indent[:-1]
        print(indent, '<', func, srcpath, line, threading.current_thread().name)


#
#
#
class Middle:
    def __init__(self, b) -> None:
        print(indent, 'b', b, ' in Base')

class A(Middle):

    def __init__(self, a=1, b=0) -> None:
        super().__init__(b)

    def memberfunc(self, a,b):
        print(indent, 'memberfunc', a,b)


def scope_test():

    print(indent, '-------------- max_count=2 test ---------------')
    def max_count_test(a,b,c):
        print(indent, f'It will shown only max_count(2) times', a,b,c)

        obj = A(100,101)
        obj.memberfunc(4,5)
        return a+b+c

    for _ in range(5):
        max_count_test(1,2,3)

    print(indent, '-------------- suppress test ---------------')

    def in_suppress_1():
        print(indent, 'Not seen: in_suppress_1')

    def suppress_following():
        print(indent, 'enter: suppress_following()')
        in_suppress_1()
        print(indent, 'exit: suppress_following: after in_suppress_1')
        return

    suppress_following()
    return


def print_numbers():
    sys.setprofile(cb_trace)

    for i in range(1, 3):
        print(indent, f"number: {i}")
        scope_test()
        time.sleep(1)
    sys.setprofile(None)

def print_letters():
    sys.setprofile(cb_trace)

    for letter in 'abcde':
        print(indent, f"Letter: {letter}")
        scope_test()
        time.sleep(1)
    sys.setprofile(None)


if __name__ == "__ipynb__":
    sys.setprofile(cb_trace)

    threads = []
    for i, fn in enumerate([print_numbers, print_letters]):
        t = threading.Thread(target=fn, name=f't{i}')
        threads.append(t)
        t.start()
        break # 

    for t in threads: t.join()
    sys.setprofile(None)


### with decorator

In [ ]:
import sys

import threading
import time
from scope_fn import scope


class Middle:
    @scope.fn
    def __init__(self, b) -> None:
        print('b', b, ' in Base')

class A(Middle):
    @scope.fn
    def __init__(self, a=1, b=0) -> None:
        super().__init__(b)

    @scope.fn
    def memberfunc(self, a,b):
        print('memberfunc', a,b)


@scope.fn
def scope_test():

    print('-------------- max_count=2 test ---------------')
    @scope.fn
    def max_count_test(a,b,c):
        print(f'It will shown only max_count(2) times', a,b,c)

        obj = A(100,101)
        obj.memberfunc(4,5)
        return a+b+c

    for _ in range(5):
        max_count_test(1,2,3)

    print('-------------- suppress test ---------------')

    @scope.fn
    def in_suppress_1():
        print('Not seen: in_suppress_1')

    @scope.fn
    def suppress_following():
        print('enter: suppress_following()')
        in_suppress_1()
        print('exit: suppress_following: after in_suppress_1')
        return
    
    suppress_following()
    return

@scope.fn
def print_numbers():
    # sys.setprofile(cb_trace)

    for i in range(1, 3):
        print(f"number: {i}")
        scope_test()
        time.sleep(1)

@scope.fn
def print_letters():
    # sys.setprofile(cb_trace)

    for letter in 'abcde':
        print(f"Letter: {letter}")
        scope_test()
        time.sleep(1)


if __name__ == "__ipynb__":

    threads = []
    for i, fn in enumerate([print_numbers, print_letters]):
        t = threading.Thread(target=fn, name=f't{i}')
        threads.append(t)
        t.start()

    for t in threads: t.join()



## deco

In [ ]:
import time

def decorator_function(the_func, called=0):
    def wrapper(*args, **kwargs):
        nonlocal called
        tick = time.time()
        result = the_func(*args, **kwargs)
        print(f"{the_func.__name__}: {called=}")
        called += 1
        return result
    return wrapper

def real_func():
    print('real_func: body')




real_func_var = decorator_function(real_func)  # 1

real_func_var()
real_func_var()

@decorator_function
def real_func2():
    print('real_func2: body')

real_func2()

In [ ]:

class decorator_class:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        tick = time.time()
        result = self.func(*args, **kwargs)
        print(f"{self.func.__name__}: exec={time.time() - tick}sec")
        return result


def real_func():
    print('real_func: called')



real_func = decorator_class(real_func)  # 1

real_func()


In [6]:
from dataclasses import dataclass
import enum

MAX_FUNC_LOGGING = 999999


@enum.unique
class Suppress(enum.Enum):
    SelfOnly = enum.auto()
    Following = enum.auto()
    SelfAndFollowing = enum.auto()


class Trace:
    _workdir = None

    Suppress = Suppress
    @dataclass
    class FuncContext:
        # id:str
        show_args:bool = False
        logging_cnt:int=MAX_FUNC_LOGGING
        suppress:Suppress = None


print(Trace.Suppress.Following)
print(Suppress)


Suppress.Following
<enum 'Suppress'>
